In [42]:
import pandas as pd

from typing import List, Dict
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_info_columns', 10000)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.2f}'.format)


In [43]:
from pathlib import Path
import sys

# 1) Where is this notebook?
notebook_dir = Path.cwd()

# 2) Climb up until you get to the folder that contains "app/"
#    parents[2] goes up from objetivo_2 → notebooks → objetivos → …
#    count how many levels from objetivo_2 to BOTS_RPA: in your case it's 8 levels
project_root = notebook_dir.parents[6]

# 3) Insert it at front of sys.path
sys.path.insert(0, str(project_root))

# 4) Now imports of "app.…" will succeed


In [44]:
from app.modules.sga.minpub.report_validator.service.objetivos.objetivo_1.objetivo_1 import validation_objetivo_1
from app.modules.sga.minpub.report_validator.service.objetivos.objetivo_2.objetivo_2 import validation_objetivo_2
from app.modules.sga.minpub.report_validator.service.objetivos.objetivo_3.objetivo_3 import validation_objetivo_3

In [45]:

from app.modules.sga.minpub.report_validator.service.objetivos.objetivo_2.word_averias_extractor import extract_averias_table
#from app.modules.sga.minpub.report_validator.service.objetivos.objetivo_2.word_info_tecnico_extractor import extract_tecnico_reports
from app.modules.sga.minpub.report_validator.service.objetivos.calculations import extract_tecnico_reports_without_hours_last_dates
#from app.modules.sga.minpub.report_validator.service.objetivos.objetivo_3.word_indisp_anexos_extractor import extract_indisponibilidad_anexos
from app.modules.sga.minpub.report_validator.service.objetivos.calculations import extract_indisponibilidad_anexos

In [46]:
from app.modules.sga.minpub.report_validator.service.objetivos.preprocessing import ( 
    preprocess_335, preprocess_380, preprocess_corte_excel,
    preprocess_df_cid_cuismp_sharepoint, preprocess_df_word_datos_averias,
    preprocess_df_word_telefonia_averias, preprocess_df_word_datos_informe_tecnico,
    preprocess_df_word_telefonia_informe_tecnico ,preprocess_df_word_datos_anexos_indis,
    preprocess_df_word_telefonia_anexos_indis,
)

In [47]:
from app.modules.sga.minpub.report_validator.service.objetivos.mergers.merge_sga_335_corte_excel_sharepoint_cuismp_sga380 import ( 
merge_sga_335_corte_excel_sharepoint_cuismp_sga380
)
from app.modules.sga.minpub.report_validator.service.objetivos.mergers.merge_word_datos_averias_corte_excel import ( 
merge_word_datos_averias_corte_excel
)
from app.modules.sga.minpub.report_validator.service.objetivos.mergers.merge_word_telefonia_averias_corte_excel import ( 
merge_word_telefonia_averias_corte_excel
)
from app.modules.sga.minpub.report_validator.service.objetivos.mergers.merge_word_datos_informe_corte_excel import ( 
merge_word_datos_informe_corte_excel
)
from app.modules.sga.minpub.report_validator.service.objetivos.mergers.merge_word_telefonia_informe_corte_excel import ( 
merge_word_telefonia_informe_corte_excel
)
from app.modules.sga.minpub.report_validator.service.objetivos.mergers.merge_word_datos_anexos_disponibilidad_dfs_merged_sga import ( 
merge_word_datos_anexos_disponibilidad_df_merged_sga
)
from app.modules.sga.minpub.report_validator.service.objetivos.mergers.merge_word_telefonia_anexos_disponibilidad_dfs_merged_sga import ( 
merge_word_telefonia_anexos_disponibilidad_df_merged_sga
)

In [48]:
BASE_DIR = Path.cwd().parent.parent.parent.parent.parent.parent.parent
SAVE_DIR_EXTRACT_EXCEL = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "excel"/ "CORTE 2 30.03.25 AL 06.04.25.xlsx"
SAVE_DIR_EXTRACT_SGA_335 = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "sga_335" / "sga_reporte_15-03-2025_06-04-2025_20250429_210214.xlsx"
CID_CUISMP_PATH = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "sharepoint_cid_cuismp" / "MINPU - CID-CUISMP - AB (1)_20250429_210918.xlsx"
DIR_PARADAS_RELOJ = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "pausa_cliente" / "sga_reporte_15-03-2025_06-04-2025_20250429_213105.xlsx"
DIR_WORD_DATOS = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "word_datos" / "COMPONENTE 2-DATOS - CORTE 2 (002).docx"
DIR_WORD_TELEFONIA = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "word_telefonia" / "COMPONENTE 4 - TELEFONOS - CORTE 2 (002).docx"

In [49]:


def all_objetivos(
    path_corte_excel, 
    path_sgq_dinamico_335, 
    path_sga_dinamico_380,
    path_cid_cuismp_sharepoint,
    word_datos_file_path,
    word_telefonia_file_path

) -> List[Dict]:
    """
    Calls each objective's validation function and combines the results.
    Each objective function returns a DataFrame with the columns:
      - 'numero de incidencia'
      - 'mensaje'
      - 'objetivo'
    
    Returns:
      A list of dictionaries (one per incident that fails at least one validation)
      across all objectives.
    """
    results = []
    
    # extract
    df_corte_excel = pd.read_excel(path_corte_excel, skipfooter=2, engine="openpyxl")
    df_sga_dinamico_335 = pd.read_excel(path_sgq_dinamico_335) 
    df_sga_dinamico_380 = pd.read_excel(path_sga_dinamico_380)
    df_cid_cuismp_sharepoint = pd.read_excel(path_cid_cuismp_sharepoint)
    df_word_datos_averias =  extract_averias_table(word_datos_file_path)
    df_word_telefonia_averias = extract_averias_table(word_telefonia_file_path)
    df_word_datos_informe_tec =  extract_tecnico_reports_without_hours_last_dates(word_datos_file_path)
    df_word_telefonia_informe_tec = extract_tecnico_reports_without_hours_last_dates(word_telefonia_file_path)
    df_word_datos_anexos_indis =  extract_indisponibilidad_anexos(word_datos_file_path)
    df_word_telefonia_anexos_indis = extract_indisponibilidad_anexos(word_telefonia_file_path)


    # preprocess
    df_word_datos_averias = preprocess_df_word_datos_averias(df_word_datos_averias)
    df_word_telefonia_averias = preprocess_df_word_telefonia_averias(df_word_telefonia_averias)
    
    df_word_datos_informe_tec =  preprocess_df_word_datos_informe_tecnico(df_word_datos_informe_tec)
    df_word_telefonia_informe_tec = preprocess_df_word_telefonia_informe_tecnico(df_word_telefonia_informe_tec)

    df_word_datos_anexos_indis = preprocess_df_word_datos_anexos_indis(df_word_datos_anexos_indis)
    df_word_telefonia_anexos_indis = preprocess_df_word_telefonia_anexos_indis(df_word_telefonia_anexos_indis)

    df_sga_dinamico_335 = preprocess_335(df_sga_dinamico_335)
    df_sga_dinamico_380 = preprocess_380(df_sga_dinamico_380)
    df_corte_excel = preprocess_corte_excel(df_corte_excel)
    df_cid_cuismp_sharepoint = preprocess_df_cid_cuismp_sharepoint(df_cid_cuismp_sharepoint)
    

    # mergers 
    #  SGA 335 - 380 - SHAREPOINT - CORTE - BOTH
    df_matched_corte_sga335_Sharepoint_cuismp_sga380 = merge_sga_335_corte_excel_sharepoint_cuismp_sga380(
        df_corte_excel, df_sga_dinamico_335,
        df_cid_cuismp_sharepoint, df_sga_dinamico_380,
        'both'
        )
    
    #  SGA 335 - 380 - SHAREPOINT - CORTE - LEFT ONLY
    df_unmatched_corte_sga335_Sharepoint_cuismp_sga380 = merge_sga_335_corte_excel_sharepoint_cuismp_sga380(
        df_corte_excel,
        df_sga_dinamico_335,
        df_cid_cuismp_sharepoint,
        df_sga_dinamico_380,
        'left_only'
        )
    
    # AVERIAS - DATOS - EXCEL
    df_matched_word_datos_averias_corte_excel = merge_word_datos_averias_corte_excel(
        df_corte_excel,
        df_word_datos_averias,
        'both'
        )
   
    # AVERIAS - TELEFONIA - EXCEL
    df_matched_word_telefonia_averias_corte_excel = merge_word_telefonia_averias_corte_excel(
        df_word_telefonia_averias,
        df_corte_excel,
        'both'
        )
    
    #INFORME TECNICO - DATOS - EXCEL
    df_matched_word_datos_informe_tecnico_corte_excel = merge_word_datos_informe_corte_excel(
        df_word_datos_informe_tec,
        df_corte_excel,
        'both'
        )
    
    #INFORME TECNICO - TELEFONIA - EXCEL
    df_matched_word_telefonia_informe_tecnico_corte_excel = merge_word_telefonia_informe_corte_excel(
        df_word_telefonia_informe_tec,
        df_corte_excel,
        'both'
        )
    

    #ANEXOS INDISPONIBILIDAD - DATOS - EXCEL
    df_matched_word_datos_anexo_indisponibilidad_df_merged_sga = merge_word_datos_anexos_disponibilidad_df_merged_sga(
        df_word_datos_anexos_indis,
        df_matched_corte_sga335_Sharepoint_cuismp_sga380,
        'both'
        )
    
    #ANEXOS INDISPONIBILIDAD - TELEFONIA - EXCEL
    df_matched_word_telefonia_anexo_indisponibilidad_df_merged_sga = merge_word_telefonia_anexos_disponibilidad_df_merged_sga(
        df_word_telefonia_anexos_indis,
        df_matched_corte_sga335_Sharepoint_cuismp_sga380,
        'both'
        )
    

    # OBJETIVOS 

    obj1_df = validation_objetivo_1(
        df_matched_corte_sga335_Sharepoint_cuismp_sga380,
        df_unmatched_corte_sga335_Sharepoint_cuismp_sga380
        )

    obj2_df = validation_objetivo_2(
        df_matched_word_datos_averias_corte_excel,
        df_matched_word_telefonia_averias_corte_excel,
        df_matched_word_datos_informe_tecnico_corte_excel,
        df_matched_word_telefonia_informe_tecnico_corte_excel
        )
    
    obj3_df = validation_objetivo_3(
        df_matched_word_datos_anexo_indisponibilidad_df_merged_sga,
        df_matched_word_telefonia_anexo_indisponibilidad_df_merged_sga
    )   

    results.extend(obj1_df.to_dict(orient='records'))
    results.extend(obj2_df.to_dict(orient='records'))
    results.extend(obj3_df.to_dict(orient='records'))

    df_all = pd.DataFrame(results)

    df_grouped = (
        df_all
        .groupby('nro_incidencia', as_index=False)
        .agg({'mensaje': lambda msgs: ' | '.join(msgs),
              'objetivo': lambda objs: ' | '.join(objs), 
              'TIPO REPORTE': 'first', 
              })
    )

    #return df_grouped.to_dict(orient='records')
    return df_grouped


result_df = all_objetivos(
    SAVE_DIR_EXTRACT_EXCEL, 
    SAVE_DIR_EXTRACT_SGA_335, 
    DIR_PARADAS_RELOJ,
    CID_CUISMP_PATH,
    DIR_WORD_DATOS,
    DIR_WORD_TELEFONIA

)

result_df




No clock stops found for incident 21790147
No clock stops found for incident 21790487
No clock stops found for incident 21791695
No clock stops found for incident 21792007
No clock stops found for incident 21792345
No clock stops found for incident 21792390
No clock stops found for incident 21792405
No clock stops found for incident 21792407
No clock stops found for incident 21792453
No clock stops found for incident 21792494
No clock stops found for incident 21792500
No clock stops found for incident 21792504
No clock stops found for incident 21792509
No clock stops found for incident 21793097
Adjusting end time to interruption en for incident 21793573
Adjusting end time to interruption en for incident 21793573
Adjusting end time to interruption en for incident 21793573
No clock stops found for incident 21795643
No clock stops found for incident 21796219
No clock stops found for incident 21796247
No clock stops found for incident 21796252
No clock stops found for incident 21796282
No 

Converted 20:30 to 1230.0 seconds
Converted 20:08 to 1208.0 seconds
Converted 03:13 to 193.0 seconds
Converted 213:02 to 12782.0 seconds
Converted 186:32 to 11192.0 seconds
Converted 120:16 to 7216.0 seconds
Converted 05:07 to 307.0 seconds
Converted 187:08 to 11228.0 seconds
Converted 330:10 to 19810.0 seconds
Converted 378:50 to 22730.0 seconds
Converted 357:45 to 21465.0 seconds
Converted 115:50 to 6950.0 seconds
Converted 26:39 to 1599.0 seconds
Converted 97:20 to 5840.0 seconds
Converted 20:10 to 1210.0 seconds
Converted 00:21 to 21.0 seconds
Converted 111:13 to 6673.0 seconds
Converted 08:59 to 539.0 seconds
Converted 42:01 to 2521.0 seconds
Converted 00:39 to 39.0 seconds
Converted 02:06 to 126.0 seconds
Converted 23:07 to 1387.0 seconds
Converted 00:54 to 54.0 seconds
Converted 06:03 to 363.0 seconds
Converted 06:52 to 412.0 seconds
Converted 15:52 to 952.0 seconds
Converted 11:29 to 689.0 seconds
Converted 11:33 to 693.0 seconds
Converted 12:13 to 733.0 seconds
Converted 06:24

,nro_incidencia,mensaje,objetivo,TIPO REPORTE
0,21789759,No coincide total horas sin acceso a la sede de word indisponibilidad en anexos : (Total de horas sin acceso a la sede: 16:22horas) es diferente a sga : (Total de horas sin acceso a la sede: 16:22 horas).,3.1,RECLAMO
1,21789943,Periodo(s) inválido(s) CORTE -EXCEL : 18/03/2025 17:24:00 hasta el día 19/03/2025 7:00:00 ES DIFERENTE A SGA PAUSA CLIENTE SIN OVERLAP: 18/03/2025 17:24:00 hasta el día 19/03/2025 07:00:00. | No coincide Determinación de la causa de WORD informe técnico : El inconveniente se originó debido a un problema con el equipo de red satelital ubicado en la sede del cliente. es diferente a DETERMINACION DE LA CAUSA de Excel: El inconveniente se originó debido a un problema con el equipo de red satelital ubicado en la sedel cliente..,1.11 | 2.2,RECLAMO
2,21790147,No coincide AVERÍA en EXCEL-CORTE: (ALERTA EN EL GESTOR DE MONITOREO - SIN AFECTACIÓN DEL SERVICIO) con tipificacion problema en SGA 335: (INCONVENIENTES EN LAS POLITICAS DE CALIDAD DE SERVICIO-RECONFIGURACIÓN).,1.6,RECLAMO
3,21790461,Periodo(s) inválido(s) CORTE -EXCEL : 19/03/2025 22:43:00 hasta el día 28/03/2025 8:00:00 ES DIFERENTE A SGA PAUSA CLIENTE SIN OVERLAP: 19/03/2025 22:43:00 hasta el día 28/03/2025 08:00:00.,1.11,RECLAMO
4,21790467,No coincide Determinacion de la causa Excel-Corte: (Durante la revisión no se encontró anexos inoperativos en la sede del cliente.) con SGA: (No se encontraron anexos inoperativos durante la revisión en la sede del cliente.). | La fecha de inicio del parrafo en it_medidas_tomadas: ( 19/03/2025 15:18 ) no coincide con la fecha inicio de la penultima fila: 18/03/2025 12:46. | Periodo(s) inválido(s) CORTE -EXCEL : 19/03/2025 15:54:00 hasta el día 27/03/2025 9:50:00 ES DIFERENTE A SGA PAUSA CLIENTE SIN OVERLAP: 19/03/2025 15:54:00 hasta el día 27/03/2025 09:50:00. | No coincide Determinación de la causa de WORD informe técnico : No se encontró anexos inoperativos en la sede del cliente. es diferente a DETERMINACION DE LA CAUSA de Excel: Durante la revisión no se encontró anexos inoperativos en la sede del cliente..,1.5 | 1.8 | 1.11 | 2.2,RECLAMO
5,21790478,Periodo(s) inválido(s) CORTE -EXCEL : 19/03/2025 16:45:00 hasta el día 20/03/2025 8:00:00\n20/03/2025 16:45:00 hasta el día 21/03/2025 8:00:00\n21/03/2025 16:45:00 hasta el día 24/03/2025 8:00:00 ES DIFERENTE A SGA PAUSA CLIENTE SIN OVERLAP: 19/03/2025 16:45:00 hasta el día 20/03/2025 08:00:00\n20/03/2025 16:45:00 hasta el día 21/03/2025 08:00:00\n21/03/2025 16:45:00 hasta el día 24/03/2025 08:00:00.,1.11,RECLAMO
6,21790487,"No coincide AVERÍA en EXCEL-CORTE: (ANEXO SIN INCONVENIENTE) con tipificacion problema en SGA 335: (ANEXO DESCONFIGURADO). | No coincide MEDIDAS CORRECTIVAS de WORD informe técnico : El cliente, el Sr. Elvis Reategui, solicita revisar los anexos telefónicos de la sede identificado con el CUISMP 28001, se generó ticket el día 19/03/2025 a las 15:37 horas. Inmediatamente, Claro se comunicó con el cliente de la sede para coordinar una ventana de trabajo. Sin embargo, el cliente Sr. Carlos Villegas, el día 19/03/2025 a las 20:44 horas, indicó que los anexos se encontraban operativos y sin problemas. Finalmente, se verificó el correcto funcionamiento y estabilidad del servicio.\n es diferente a MEDIDAS CORRECTIVAS de Excel: El cliente, el Sr. Elvis Reategui, solicita revisar los anexos telefonicos de la sede identificado con el CUISMP 28001, se generó ticket el día 19/03/2025 a las 15:37 horas. Inmediatamente, Claro se comunicó con el cliente de la sede para coordinar una ventana de trabajo. Sin embargo, el cliente Sr. Carlos Villegas, el día 19/03/2025 a las 20:44 horas, indicó que los anexos se encontraban operativos y sin problemas. Finalmente, se verificó el correcto funcionamiento y estabilidad del servicio..",1.6 | 2.2,RECLAMO
7,21790524,Periodo(s) inválido(s) CORTE -EXCEL : 19/03/2025 22:53:00 hasta el día 27/03/2025 9:00:00 ES DIFERENTE A SGA PAUSA CLIENTE SIN OVERLAP: 19/03/2025 22:53:0

In [50]:
from datetime import datetime
BASE_DIR = Path.cwd().parent.parent.parent.parent.parent.parent.parent
timestamp = datetime.now().strftime('%Y-%m-%d_%d_%H%M')
SAVE_FINAL_PATH = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "final" / f"validacion_total_{timestamp}.xlsx"

result_df.to_excel(SAVE_FINAL_PATH, index=False)
print(f"file saved")

file saved
